In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

2025-08-17 14:00:06.973521: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-17 14:00:07.073404: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-17 14:00:13.131648: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
train_data = pd.read_csv('datass/Files/train_emoji.csv')
test_data = pd.read_csv('datass/Files/tesss.csv')

In [3]:
X_train = train_data['never talk to me again']
y_train = train_data['3']
X_train[131] = 'never talk to me again'
y_train[131] = 3

/tmp/ipykernel_39627/3497168708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[131] = 'never talk to me again'
/tmp/ipykernel_39627/3497168708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train[131] = 3


In [4]:
X_test = test_data['I want to eat\t'].map(lambda x:x.replace("\t",""))
y_test = test_data['4']

In [5]:
def convert_to_one_hot(y_train , depth = 5):
    one_hot_ds = []
    for i in range(len(y_train)):
        temp = [0]*depth
        temp[y_train[i]]+=1
        one_hot_ds.append(temp)
    np_array = np.array(one_hot_ds)
    return np_array

In [6]:
with open("datass/Files/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    word_to_vec_map = {}
    word_to_index_map = {}
    index_to_word_map = {}
    index = 1
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_to_vec_map[word] = vector
        word_to_index_map[word] = index
        index_to_word_map[index] = word
        index+=1

In [7]:
# def sentence_to_indices(sentence):
#     sentenceArr = sentence.lower().split()
#     sentenceArr = [word_to_index_map[x] for x in sentenceArr]
#     return sentenceArr

In [8]:
def sentence_to_indices(sentence,maxLength = 10):
    m = sentence.shape[0]
    finalArr = np.zeros((m,maxLength),dtype = 'int')
    for i in range(m):
        ind = 0
        for j in sentence[i].lower().split():
            if j in word_to_index_map.keys():
                finalArr[i][ind] = word_to_index_map[j]
                ind+=1
    return finalArr

In [9]:
sentArr = pd.array(["hello","how are you","wassup my boy"])
sentence_to_indices(sentArr)

array([[ 13076,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [   198,     33,     82,      0,      0,      0,      0,      0,
             0,      0],
       [243124,    193,   1607,      0,      0,      0,      0,      0,
             0,      0]])

In [10]:
def pretrained_embedding_layer(word_to_vec_map , index_to_word_map):
    vocab_size = len(word_to_vec_map.keys()) + 1
    random_word = word_to_vec_map['random']
    
    emb_matrix = np.zeros((vocab_size , random_word.shape[0]))
    for idx,word in index_to_word_map.items():
        emb_matrix[idx,:] = word_to_vec_map[word]
    
    embedding_layer = tf.keras.layers.Embedding(vocab_size , random_word.shape[0],trainable=False)
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [11]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map , index_to_word_map)

E0000 00:00:1755419437.594421   39627 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1755419437.609389   39627 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
def EmojifyV2():
    sentence_indices = tf.keras.layers.Input(shape = (10,),dtype = 'int32')
    embeddings = embedding_layer(sentence_indices)
    X = tf.keras.layers.LSTM(units = 128 , return_sequences = True)(embeddings)
    X = tf.keras.layers.Dropout(rate = 0.5)(X)
    X = tf.keras.layers.LSTM(units = 128)(embeddings)
    X = tf.keras.layers.Dropout(rate = 0.5)(X)
    X = tf.keras.layers.Dense(units = 5)(X)
    X = tf.keras.layers.Activation('softmax')(X)
    model = tf.keras.Model(inputs = sentence_indices , outputs = X)
    return model

In [13]:
model =  EmojifyV2()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 10, 50)         │    20,000,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,092,343 (76.65 MB)

 Trainable params: 92,293 (360.52 KB)

 Non-trainable params: 20,000,050 (76.29 MB)

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
X_train_indices =sentence_to_indices(X_train) 
y_train_oh = convert_to_one_hot(y_train)
X_test_indices =sentence_to_indices(X_test) 
y_test_oh = convert_to_one_hot(y_test)

In [16]:
y_train_oh

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0,

In [ ]:
model.fit(X_train_indices , y_train_oh , epochs = 50, batch_size = 2)


Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3030 - loss: 1.5737
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4924 - loss: 1.2938
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5530 - loss: 1.0084
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7045 - loss: 0.8169
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7576 - loss: 0.6418
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8182 - loss: 0.4711
Epoch 7/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8106 - loss: 0.5346
Epoch 8/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8712 - loss: 0.3375
Epoch 9/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8636 - loss: 0.2861
Epoch 10/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9394 - loss: 0.2102
Epoch 11/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8788 - loss: 0.2953
Epoch 12/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9697 - 

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=saved_model/my_model.

In [23]:
model.save('my_model.keras')


In [18]:
predArr = sentence_to_indices(np.array(["I hate you"]))

In [19]:
model.evaluate(X_test_indices,y_test_oh)
pred = model.predict(predArr)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.8727 - loss: 0.8499 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


In [184]:
label_to_emoji = {0:"❤️",1:"⚾",2:"😀",3:"😔",4:"🍴"}

In [203]:
def predict_emoji(predArr,predictions):
    for i in range(len(predictions)):
        print(f"{predArr[i]} {label_to_emoji[np.argmax(predictions[i])]}")

In [207]:
origArr = ["I hate you","I love you"]
predArr = sentence_to_indices(np.array(["I hate you","I love you"]))

In [208]:
predictions = model.predict(predArr)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [209]:
predict_emoji(origArr,predictions)

I hate you 😔
I love you ❤️
